In [1]:
print("Ok")

Ok


In [2]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader
import pinecone
from langchain.prompts import PromptTemplate

/home/codespace/.python/current/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Create pinecone cluster, get API key

In [3]:
PINECONE_API_KEY = "37d70112-c425-4c0b-80bd-846d1429eaf9"
PINECONE_API_ENV = ""

Extract pdf data

In [4]:

from langchain.document_loaders import PyPDFLoader
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf("data/")

FileNotFoundError: Directory not found: 'data/'

In [ ]:
# extracted_data

Create text chunks

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("Number of ch chunks is", len(text_chunks))

Download embedding model

In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
embeddings

Test embedding model

In [ ]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Vector representation of "Hello World"

In [ ]:
query_result

Initializing Pinecone

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "mnnit-chatbot"

Creating embeddings for each text chunk and storing in the vector database

In [ ]:
from langchain_pinecone import PineconeVectorStore
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
docsearch = PineconeVectorStore.from_documents(
    text_chunks,
    embeddings,
    index_name = index_name
)

Trying out the index

In [ ]:
# # docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = "What is the NIRF rank of MNNIT?"
# query_vector = embeddings.embed_query(query)
index = pc.Index(index_name)
# results = index.query(vector = query_vector, top_k = 3, include_values = True)
# #docs = docsearch.similarity_search(query, k = 3)
# print(results)


from langchain_pinecone import PineconeVectorStore
vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

results = vectorstore.similarity_search(
    query,
    k=3  # return 3 most relevant docs 
)

print(results)

Load llm model

In [ ]:
import langchain
import langchain_community
from langchain_community.llms import CTransformers
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type = "llama",
                    verbose = True,
                    config = {'max_new_tokens':512,
                              'temperature':0.8})

Entering LLM Prompt Template

In [ ]:
print(llm)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Don't answer any queries not related to MNNIT.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
from langchain.chains import RetrievalQA
qa=RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever = vectorstore.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query":user_input})
    print("Response: ",result["result"])